In [ ]:
# 1. 필요한 라이브러리 설치
!pip install openai==0.28 transformers

In [ ]:
# 2. 라이브러리 불러오기
import openai
import random
import json
import time
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from sentence_transformers import SentenceTransformer, util
from google.colab import files

In [ ]:
# 3. OpenAI API 키 설정
openai.api_key = "***"

In [ ]:
# 4. JSON 데이터 업로드
uploaded = files.upload()
with open("knowledge.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
# 5. GPT 번역 함수
def gpt_translate(text, source_lang="Korean", target_lang="English"):
    prompt = f"Translate the following text from {source_lang} to {target_lang}:\n\n{text}"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful translation assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=512
        )
        translated = response['choices'][0]['message']['content'].strip()
        return translated
    except Exception as e:
        print(f"[ERROR] GPT translation failed: {e}")
        return None

In [ ]:
# 6. T5 패러프레이즈 모델 로딩
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
paraphraser = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_paraphrases(text, num_return_sequences=3):
    input_text = f"paraphrase: {text}"
    outputs = paraphraser(
        input_text,
        max_length=100,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=100,
        top_p=0.92,
        temperature=0.8
    )
    return [output['generated_text'] for output in outputs]

In [ ]:
# 7. 동의어 사전 및 치환 함수
synonym_dict = {
    "소설": ["소설책", "문학작품"],
    "주인공": ["주 등장인물", "주연 인물"],
    "살고 있던": ["거주하던", "거주하고 있던"],
    "소행성": ["천체", "작은 천체"],
    "이름": ["명칭", "호칭"],
    "일하는": ["근무하는", "소속된"],
    "부서": ["부문", "팀"],
    "사랑에 빠진": ["사랑하게 된", "연인으로 맺어진"],
    "범죄를 저지른": ["죄를 범한", "죄를 저지른"],
    "이유": ["원인", "동기"],
    "약자": ["머리글자", "이니셜"],
    "가장 큰": ["최대의", "가장 거대한"],
    "섬": ["도서", "섬 지역"],
    "호수": ["호", "호수 지역"],
    "강": ["하천", "강물"],
    "용어": ["단어", "표현"],
    "뜻하는": ["의미하는", "나타내는"],
    "단위": ["측정 단위", "단위 수"],
    "몇": ["얼마나", "어느 정도"],
    "무엇일까요": ["어떤 것일까요", "무엇일까"],
}

def synonym_replace(korean_text):
    if not korean_text:
        return ""
    words = korean_text.split()
    new_words = []
    for word in words:
        for key, synonyms in synonym_dict.items():
            if key in word and random.random() < 0.5:
                word = word.replace(key, random.choice(synonyms))
                break
        new_words.append(word)
    return ' '.join(new_words)

In [ ]:
# 8. 스타일 변환 함수
style_templates = [
        "다음을 구하시오. {문장}",
        "문제를 잘 읽고 답을 구하시오. {문장}",
        "{문장} 이 문제를 해결해 보세요.",
        "문제 드립니다. {문장}",
        "생각해 보세요. {문장}",
    ]

def style_transfer(text, apply_prob=0.6):
    if random.random() > apply_prob:
        return text

    template = random.choice(style_templates)
    styled_text = template.replace("{문장}", text.strip())

    return styled_text

In [ ]:
# 9. 의미 보존 함수
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

def filter_meaning_preserved(original, candidates, threshold):
    original_emb = embed_model.encode(original, convert_to_tensor=True)
    results = []
    for c in candidates:
        c_emb = embed_model.encode(c, convert_to_tensor=True)
        sim = util.pytorch_cos_sim(original_emb, c_emb).item()
        if sim >= threshold:
            results.append(c)
    return results

In [ ]:
# 10. 유사문장 제거
def deduplicate_by_similarity(sentences, threshold):
    unique = []
    embeddings = embed_model.encode(sentences, convert_to_tensor=True)

    for i, sent in enumerate(sentences):
        is_duplicate = False
        for u_idx in range(len(unique)):
            sim = util.pytorch_cos_sim(embeddings[i], embeddings[sentences.index(unique[u_idx])]).item()
            if sim >= threshold:
                is_duplicate = True
                break
        if not is_duplicate:
            unique.append(sent)
    return unique

In [ ]:
# 11. 중복 제거된 문장 찾기
def is_similar_to_any(candidate, existing_list, threshold):
    candidate_embed = embed_model.encode(candidate, convert_to_tensor=True)
    for text in existing_list:
        sim = util.pytorch_cos_sim(candidate_embed, embed_model.encode(text, convert_to_tensor=True)).item()
        if sim >= threshold:
            return True
    return False

In [ ]:
# 12. 의미 보존 + 중복 제거
def enforce_min_questions(original, candidates, min_count):
    preserved = filter_meaning_preserved(original, candidates, threshold=0.75)
    deduped = deduplicate_by_similarity(preserved, threshold=0.92)

    if len(deduped) < min_count:
        rejected = [p for p in preserved if p not in deduped and not is_similar_to_any(p, deduped, threshold=0.92)]
        rejected_sorted = sorted(
            rejected,
            key=lambda x: -util.pytorch_cos_sim(
                embed_model.encode(original, convert_to_tensor=True),
                embed_model.encode(x, convert_to_tensor=True)
            ).item()
        )

        for r in rejected_sorted:
            deduped.append(r)
            if len(deduped) >= min_count:
                break

    return deduped

In [ ]:
# 13. 전체 백번역 증강 파이프라인
def full_back_translate_with_paraphrase_style(korean_text, num_augments):
    en_text = gpt_translate(korean_text, source_lang='Korean', target_lang='English')
    if not en_text:
        return [korean_text]

    paraphrases = generate_paraphrases(en_text, num_return_sequences=num_augments)

    results = []
    for p in paraphrases:
        ko_text = gpt_translate(p, source_lang='English', target_lang='Korean')
        if not ko_text:
            ko_text = korean_text
        ko_text_synonym = synonym_replace(ko_text)
        final_text = style_transfer(ko_text_synonym)
        results.append(final_text)

    return results

In [ ]:
# 14. 증강 실행 및 저장
augmented_data = []
for item in data:
    question = item['Question']
    answer = item['Answer']

    augmented_questions = full_back_translate_with_paraphrase_style(question, num_augments=6)
    final_questions = enforce_min_questions(question, augmented_questions, 4)

    for aug_q in final_questions:
        augmented_data.append({
            "Question": aug_q,
            "Answer": answer
        })

In [ ]:
# 15. JSON 저장 및 다운로드
output_filename = "augmented_knowledge.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)

files.download(output_filename)